In [17]:
import nevergrad as ng
import numpy as np
import matplotlib.pyplot as plt

### Problem Formulation

The financial portfolio optimization problem is a classic challenge in finance that involves determining the optimal allocation of assets within an investment portfolio. The primary goal is to construct a portfolio that achieves the best possible trade-off between expected returns and risk, typically measured as volatility. Investors seek to maximize returns while minimizing the overall portfolio risk.

\begin{align*}
\text{Maximize: } & \sum_{i=1}^{N} w_i \cdot \text{expected\_returns}_i \\
\text{Minimize: } & \sqrt{\sum_{i=1}^{N} \sum_{j=1}^{N} w_i \cdot w_j \cdot \text{covariance\_matrix}_{ij}}
\end{align*}

### Sample Data

In [18]:
# Sample data
expected_returns_simulated = np.array([0.08, 0.12, 0.10, 0.15, 0.09])
covariance_matrix_simulated = np.array([[0.0004, 0.0003, 0.0002, 0.0001, 0.0002],
                                        [0.0003, 0.0009, 0.0004, 0.0002, 0.0003],
                                        [0.0002, 0.0004, 0.0010, 0.0003, 0.0004],
                                        [0.0001, 0.0002, 0.0003, 0.0008, 0.0002],
                                        [0.0002, 0.0003, 0.0004, 0.0002, 0.0005]])

### Solving the Bi-objective Problem Using Nevergrad

In [19]:
# Bi-objective problem: Financial portfolio optimization
def portfolio_optimization(weights, expected_returns, covariance_matrix):
    portfolio_return = np.dot(weights, expected_returns)
    portfolio_volatility = np.sqrt(np.dot(weights, np.dot(covariance_matrix, weights)))
    # maximize portfolio return and minimize portfolio volatility
    return [-portfolio_return, portfolio_volatility]

In [20]:
# Using Nevergrad to solve the bi-objective problem
optimizer = ng.optimizers.CMA(parametrization= 5, budget=100)

initial_weights = np.random.dirichlet(np.ones(5), size=1).flatten()  # Random initial weights

optimizer.tell(ng.p.Array(init=initial_weights.tolist()), [0, 1],
               portfolio_optimization(initial_weights, expected_returns_simulated, covariance_matrix_simulated))

# Minimize the bi-objective problem
optimizer.minimize(lambda x: portfolio_optimization(x, expected_returns_simulated, covariance_matrix_simulated), verbosity=2, constraint_violation=[lambda x: -np.sum(x) + 1, lambda x: np.min(x) - 0])

print("Pareto front:")
for param in sorted(optimizer.pareto_front(), key=lambda p: p.losses[0]):
    print(f"{param} with losses {param.losses}")

Launching 1 jobs with new suggestions
Updating fitness with value [0.4964075673210142, 0.09897352727609786]
99 remaining budget and 0 running jobs
Current pessimistic best is: MultiValue<mean: 15272.545135323357, count: 1, parameter: Array{(5,)}:[0.07056221 0.39287958 0.4291642  0.10624294 0.00115108]>
Launching 1 jobs with new suggestions
Updating fitness with value [-0.07011577150879936, 0.04622760337254315]
98 remaining budget and 0 running jobs
Current pessimistic best is: MultiValue<mean: 15272.545135323357, count: 1, parameter: Array{(5,)}:[0.07056221 0.39287958 0.4291642  0.10624294 0.00115108]>
Launching 1 jobs with new suggestions
Updating fitness with value [0.32411071730458585, 0.07685465844876893]
97 remaining budget and 0 running jobs
Current pessimistic best is: MultiValue<mean: 15272.545135323357, count: 1, parameter: Array{(5,)}:[0.07056221 0.39287958 0.4291642  0.10624294 0.00115108]>
Launching 1 jobs with new suggestions
Updating fitness with value [-0.109393607359896

### Solving the Bi-objective Problem Using Raytune and Nevergrad

In [21]:
import time
import ray
import nevergrad as ng
from ray import train, tune
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.nevergrad import NevergradSearch

In [22]:
def evaluate(weight_1, weight_2, weight_3, weight_4, weight_5, alpha1, alpha2):
    weights = [weight_1, weight_2, weight_3, weight_4, weight_5]

    portfolio_return = sum(weights[i] * expected_returns_simulated[i] for i in range(5))

    portfolio_volatility = sum(
        weights[i] * weights[j] * covariance_matrix_simulated[i, j] 
        for i in range(5) for j in range(5)
    )

    portfolio_volatility = np.sqrt(portfolio_volatility)
    
    return -alpha1 * portfolio_return + alpha2 * portfolio_volatility

def objective(config):
    alpha1 = config["alpha1"]
    alpha2 = 1 - alpha1
    config["alpha2"] = alpha2

    weights = [config["weight_1"], config["weight_2"], config["weight_3"], config["weight_4"], config["weight_5"]]
    total_weight = sum(weights)
    normalized_weights = [w / total_weight for w in weights]
    config['weight_1'] = normalized_weights[0]
    config['weight_2'] = normalized_weights[1]
    config['weight_3'] = normalized_weights[2]
    config['weight_4'] = normalized_weights[3]
    config['weight_5'] = normalized_weights[4]

    score = evaluate(config["weight_1"], config["weight_2"], config["weight_3"], config["weight_4"], config["weight_5"], alpha1, alpha2)
    train.report({"mean_loss": score})

algo = NevergradSearch(
    optimizer=ng.optimizers.CMA,
)

algo = tune.search.ConcurrencyLimiter(algo, max_concurrent=4)

num_samples = 100

search_config = {
    "weight_1": tune.uniform(0, 1),
    "weight_2": tune.uniform(0, 1),
    "weight_3": tune.uniform(0, 1),
    "weight_4": tune.uniform(0, 1),
    "weight_5": tune.uniform(0, 1),
    "alpha1": tune.uniform(0, 1),
    "alpha2": 0
}

tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        num_samples=num_samples,
    ),
    param_space=search_config,
)
results = tuner.fit()

best_result_config = results.get_best_result().config
formatted_config = {key: round(value, 2) for key, value in best_result_config.items()}
print("Best results found were: ", formatted_config)

(bundle_reservation_check_func pid=13988) Traceback (most recent call last):
(bundle_reservation_check_func pid=13988)   File "python\ray\_raylet.pyx", line 1960, in ray._raylet.execute_task_with_cancellation_handler
(bundle_reservation_check_func pid=13988)   File "python\ray\_raylet.pyx", line 1617, in ray._raylet.execute_task
(bundle_reservation_check_func pid=13988)   File "python\ray\_raylet.pyx", line 1618, in ray._raylet.execute_task
(bundle_reservation_check_func pid=13988)   File "python\ray\_raylet.pyx", line 1621, in ray._raylet.execute_task
(bundle_reservation_check_func pid=13988)   File "python\ray\_raylet.pyx", line 1649, in ray._raylet.execute_task
(bundle_reservation_check_func pid=13988)   File "d:\Python\lib\site-packages\ray\_private\utils.py", line 1858, in __exit__
(bundle_reservation_check_func pid=13988)     raise KeyboardInterrupt
(bundle_reservation_check_func pid=13988) KeyboardInterrupt
(bundle_reservation_check_func pid=13988) 
(bundle_reservation_check_fun

Best results found were:  {'weight_1': 0.31, 'weight_2': 0.17, 'weight_3': 0.13, 'weight_4': 0.34, 'weight_5': 0.05, 'alpha1': 0.99, 'alpha2': 0.01}
